In [27]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mlflow
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
import joblib
import os
from dotenv import dotenv_values

In [28]:
config = dotenv_values(".env")

print(mlflow.__version__)
import sys
print(sys.executable)

python-dotenv could not parse statement starting at line 1


3.8.1
/Users/urkarsh.kulshrestha/Documents/AI_environment/work_env/appletree_end_to_end_forecasting/appletree_mlops_env/bin/python


In [31]:
'Read credentials to download data'

bucket_name = config.get("bucket_name")
org_name = config.get("org_name")
url = config.get("soracom_url")
influx_token = config.get("influx_token")
print(bucket_name, org_name, url)
client = influxdb_client.InfluxDBClient(url=url, token=influx_token, org=org_name)
query_api = client.query_api()

PXC PXC http://54-199-140-124.napter.soracom.io:47798


In [37]:
fields = [
    "actualNetLoad"
]

fields_filter = " or ".join([f'r["_field"] == "{field}"' for field in fields])

query = f'''
from(bucket: "{bucket_name}")
  |> range(start:0, stop: 2026-01-16T00:00:00Z)
  |> filter(fn: (r) => 
      r["_measurement"] == "FncPvSc"
      )
  |> filter(fn: (r) => {fields_filter})
  |> aggregateWindow(every: 60m, fn: mean, createEmpty: false)
  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
'''




tables = query_api.query_data_frame(query=query, org=org_name)


       result  table                    _start                     _stop  \
0     _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
1     _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
2     _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
3     _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
4     _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
...       ...    ...                       ...                       ...   
7332  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
7333  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
7334  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
7335  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
7336  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   

                         _time _measurement  actualNetLoad  
0    2025-03-14 12:00:00+0

In [41]:
print(tables.head(5))
print(tables.tail(5))

    result  table                    _start                     _stop  \
0  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
1  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
2  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
3  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
4  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   

                      _time _measurement  actualNetLoad  
0 2025-03-14 12:00:00+00:00      FncPvSc      31.005559  
1 2025-03-14 13:00:00+00:00      FncPvSc      22.756666  
2 2025-03-14 14:00:00+00:00      FncPvSc       8.218262  
3 2025-03-14 15:00:00+00:00      FncPvSc       6.962737  
4 2025-03-14 16:00:00+00:00      FncPvSc       6.938568  
       result  table                    _start                     _stop  \
7332  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00:00:00+00:00   
7333  _result      0 1970-01-01 00:00:00+00:00 2026-01-16 00: